In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_empirical_variable_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "a3tq7unb", 
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = ['MultiFidelityMaxValueEntropy']
target_functions = ["PestControl"]

In [3]:
grouped_runs = {(a,t): load_empirical_variable_cost_runs(a,t) for a in acquisition_functions for t in target_functions}

100%|██████████| 16/16 [00:11<00:00,  1.44it/s]


In [4]:
for a in acquisition_functions:
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T

        print(a, t, best_observed_per_seed.shape)

        best_observed_per_seed = -best_observed_per_seed
        
        best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
        best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
        best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)

        np.savetxt(f"results/empirical_cost_aware/Empirical_VariableCost_{t}_{a}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

MultiFidelityMaxValueEntropy PestControl (801, 16)
